- file path
- only 71 tweets

In [2]:
import tweepy
import datetime
import json
import sys

In [3]:
API_key_secret = ...
API_key = ...
access_token = ...
access_token_secret = ...

auth = tweepy.OAuthHandler(API_key, API_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit_notify=True)


In [4]:
query = (...)
lat = ...
long = ...
radius = #geocode=f"{lat},{long},{radius}km"
tweet_count = # total number of tweets
tweets_per_file = 5000 # number of tweets per file
count=0

In [55]:
def save_tweets(tweets, count, count_tweets, tweets_per_file):
    file_name = int(count)
    temp = count*tweets_per_file
    count_tweets = temp+tweets_per_file
    if len(tweets)>= count_tweets:
        obj = tweets[temp:(temp+tweets_per_file)]
        with open(f'output_scrape_{file_name}.json', 'w') as fout: #set indent to 0 if don't need human readable
            json.dump(obj, fout, indent=4)
    elif len(tweets)< count_tweets:
        obj = tweets[temp:]
        with open(f'output_scrape{file_name}.json', 'w') as fout:
            json.dump(obj, fout, indent=4)
    if len(tweets)> count_tweets:
        save_tweets(tweets, count+1, count_tweets, tweets_per_file)

In [56]:
tweets = []
try:
    for status in tweepy.Cursor(api.search, tweet_mode='extended', q=query).items(tweet_count):
        if (status.lang != 'en') : continue
        tweets.append({"text": status.full_text,
                       "hashtags": [d['text'] for d in status.entities['hashtags']],
                       "username": status.user.name,
                       "timestamp": status.created_at.strftime('%H:%M:%S %d/%m/%Y'),
                       "location": status.coordinates["coordinates"] if status.coordinates is not None
                         else '', 
                       "replied_to": status.in_reply_to_screen_name if status.in_reply_to_screen_name
                         is not None else '',
                       "language": status.lang, 
                       "verified": status.user.verified,
                       "followers": status.user.followers_count,
                       "friends": status.user.friends_count,
                       "statuses": status.user.statuses_count, 
                       "retweeted": status.retweet_count})
except BaseException as e:
    print('Error: ',str(e))

In [57]:
save_tweets(tweets, count, 0, tweets_per_file)